In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train1 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_activity.csv', engine = 'python')
train2 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_combat.csv', engine = 'python')
train3 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_pledge.csv', engine = 'python')
train4 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_payment.csv', engine = 'python')
train5 = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_trade.csv', engine = 'python')
y_train = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_label.csv', engine = 'python')

In [3]:
# 잔존 고객 0, 이탈 고객 1
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# 결제 기록이 없으면 0, 결제 기록이 있으면 1
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [6]:
len(strat_y_train), len(strat_y_test)

(30000, 10000)

In [7]:
strat_y_train['leave'].mean(), strat_y_test['leave'].mean()

(0.4501, 0.4501)

In [8]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

In [9]:
train_id = y_train['acc_id'].values
test_id = y_test['acc_id'].values

In [10]:
pd.DataFrame(y_train['acc_id']).to_csv('train_index.csv', index = False)
pd.DataFrame(y_test['acc_id']).to_csv('test_index.csv', index = False)

# 2. 모델링

In [11]:
X_train = pd.read_csv('X_train_ver2.csv')

In [12]:
X_train.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,target_type_1_week3_etc,target_type_1_week4_etc,source_weapon,target_weapon,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell
0,2.0,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0,0,1,0,0,0,0,0
1,5.0,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,0.162887,2.763336,...,0.000000e+00,0.000000,0,0,0,0,0,0,0,0
2,8.0,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,1.444694,1.594739,...,0.000000e+00,0.000000,0,0,0,0,0,0,1,0
3,17.0,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,1.362575,1.834339,...,7.190952e-08,0.000029,1,1,0,1,1,1,0,1
4,21.0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,0.654252,1.135141,...,0.000000e+00,0.000000,1,0,0,1,0,1,0,0


**feature**
- 범주형 : 'server', 'class', 'weapon' 거래 여부, 'armor' 거래 여부, 'accessory' 거래 여부, 'spell' 거래 여부            
- 연속형 : 그 외('acc_id' 제외)     

**target**
- 잔존 여부 : 'leave', 'survival_time'                     
- 결제 여부 : 'amount', 'amount_spent'

In [13]:
X_train_columns = X_train.columns

### 정규화(MinMax)

In [14]:
from sklearn.preprocessing import MinMaxScaler

minMaxScaler = MinMaxScaler()
# X_train = minMaxScaler.fit_transform(X_train.iloc[:, 1:])

X_train = X_train.iloc[:, 1:]

## 생존 여부

### AdaBoost

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state = 42)

param_grid = [
    {'max_depth' : [7, 8, 9, 10]}
  ]

grid_search = GridSearchCV(tree, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['leave'])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  20 | elapsed:    3.1s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.4s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1, param_grid=[{'max_depth': [7, 8, 9, 10]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=

In [16]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7161
{'max_depth': 9}


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 9), random_state = 42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'n_estimators' : [300, 500, 750]}
  ]

grid_search = GridSearchCV(ada, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['leave'])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  45 | elapsed:  5.9min remaining: 47.0min
[Parallel(n_jobs=-1)]: Done  28 out of  45 | elapsed: 13.2min remaining:  8.0min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 19.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                                criterion='gini',
                                                                                max_depth=9,
                                                                                max_features=None,
                                                                                max_leaf_nodes=None,
                                                                                min_impurity_decrease=0.0,
                                                                                min_impurity_split=None,
                                                                                min_samples_leaf=1,
                                                                                min

In [18]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7512666666666666
{'learning_rate': 0.1, 'n_estimators': 750}


In [19]:
from sklearn.externals import joblib

# ada_leave_clf = AdaBoostClassifier(random_state = 42, base_estimator = DecisionTreeClassifier(max_depth = 9),
#                                    n_estimators = 500, learning_rate = 0.001)
# ada_leave_clf.fit(X_train, y_train['leave'])
ada_leave_clf = grid_search.best_estimator_
joblib.dump(ada_leave_clf, './빅콘테스트/model/version2/ada_leave_clf_version2.pkl')

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['./빅콘테스트/model/version2/ada_leave_clf_version2.pkl']

### GradientBoost

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01],
     'max_depth' : [8, 9],
     'n_estimators' : [300, 500]}
  ]

grid_search = GridSearchCV(gb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['leave'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done  19 out of  40 | elapsed: 12.6min remaining: 13.9min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 20.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 20.7min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort=

In [21]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7513333333333333
{'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 500}


In [22]:
from sklearn.externals import joblib

# gb_leave_clf = GradientBoostingClassifier(random_state = 42, learning_rate = 0.001, max_depth = 9, n_estimators = 300)
# gb_leave_clf.fit(X_train, y_train['leave'])
gb_leave_clf = grid_search.best_estimator_
joblib.dump(gb_leave_clf, './빅콘테스트/model/version2/gb_leave_clf_version2.pkl')

['./빅콘테스트/model/version2/gb_leave_clf_version2.pkl']

### RandomForest

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['leave'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.7min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [24]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7616
{'max_features': 14, 'n_estimators': 750}


In [25]:
from sklearn.externals import joblib

# rf_leave_clf = RandomForestClassifier(random_state = 42, max_features = 19, n_estimators = 750)
# rf_leave_clf.fit(X_train, y_train['leave'])
rf_leave_clf = grid_search.best_estimator_
joblib.dump(rf_leave_clf, './빅콘테스트/model/version2/rf_leave_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_leave_clf_version2.pkl']

In [26]:
Importance = pd.DataFrame({'feature' : X_train_columns[1:], 'importance' : rf_leave_clf.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
49,enchant_count_1,0.000059
50,enchant_count_2,0.000084
52,enchant_count_4,0.000188
85,server_group_2,0.000232
51,enchant_count_3,0.000253
83,server_group_0,0.000281
84,server_group_1,0.000397
119,target_type_1_week2_enchant_scroll,0.000500
71,combat_same_pledge_cnt_3,0.000530
112,source_type_1_week3_enchant_scroll,0.000545


### XGBoost

In [27]:
# 버전에 따라 eta 또는 learing_rate

from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train, y_train['leave'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  4.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bytree=1,
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_job

In [28]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7553333333333333
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 10}


In [29]:
from sklearn.externals import joblib

# xgb_leave_clf = XGBClassifier(random_state = 42, eta = 0.1, gamma = 1, max_depth = 11)
# xgb_leave_clf.fit(X_train, y_train['leave'])
xgb_leave_clf = grid_search.best_estimator_
joblib.dump(xgb_leave_clf, './빅콘테스트/model/version2/xgb_leave_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_leave_clf_version2.pkl']

In [30]:
Importance = pd.DataFrame({'feature' : X_train_columns[1:], 'importance' : xgb_leave_clf.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
49,enchant_count_1,0.000000
50,enchant_count_2,0.000000
84,server_group_1,0.000089
51,enchant_count_3,0.000133
83,server_group_0,0.000178
92,class_6,0.000222
36,exp_recovery_4,0.000267
72,combat_same_pledge_cnt_4,0.000311
71,combat_same_pledge_cnt_3,0.000355
70,combat_same_pledge_cnt_2,0.000355


### Ensemble

In [34]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

voting_leave_clf_1 = VotingClassifier(
    estimators=[('ada', ada_leave_clf), ('gb', gb_leave_clf), ('rf', rf_leave_clf), ('xgb', xgb_leave_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_leave_clf_1, X_train,
                           y_train['leave'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.7602004668842722

In [35]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

voting_leave_clf_2 = VotingClassifier(
    estimators=[('rf', rf_leave_clf), ('xgb', xgb_leave_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_leave_clf_2, X_train,
                           y_train['leave'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.7606334335675953

In [36]:
voting_leave_clf_1.fit(X_train, y_train['leave'])
joblib.dump(voting_leave_clf_1, './빅콘테스트/model/version2/voting_leave_clf_1_version2.pkl')

['./빅콘테스트/model/version2/voting_leave_clf_1_version2.pkl']

In [37]:
voting_leave_clf_2.fit(X_train, y_train['leave'])
joblib.dump(voting_leave_clf_2, './빅콘테스트/model/version2/voting_leave_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_leave_clf_2_version2.pkl']

### MLP(Multi Layer Perceptron)

In [38]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

input = Input(shape = (X_train.shape[1], ))

fc1 = Dense(1024, activation='relu')(input)
fc2 = Dense(512, activation='relu')(fc1)
fc3 = Dense(256, activation='relu')(fc2)
fc4 = Dense(128, activation='relu')(fc3)
fc4_drop = Dropout(0.1)(fc4)
fc5 = Dense(64, activation='relu')(fc4_drop)
fc5_drop = Dropout(0.1)(fc5)
fc6 = Dense(32, activation='relu')(fc5_drop)

leave_out = Dense(1, activation='sigmoid', name='leave_output')(fc6)

mlp_leave_clf = Model(inputs = input, outputs = leave_out)
mlp_leave_clf.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 146)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              150528    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
__________

In [39]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

model_path = './빅콘테스트/model/version2/mlp_leave_clf.hdf5'

mlp_leave_clf.compile(optimizer = 'rmsprop', loss = {'leave_output' : 'binary_crossentropy'},
              metrics = {'leave_output' : 'acc'})

callbacks_list = [EarlyStopping(monitor = 'val_loss', patience = 20),
                 ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10),
                 ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)]

history = mlp_leave_clf.fit(X_train, y_train['leave'],
                            epochs = 150, batch_size = 128, 
                            callbacks = callbacks_list,
                            validation_split = 0.2,
                            shuffle = True)

Train on 24000 samples, validate on 6000 samples
Epoch 1/150
24000/24000 [==============================] - 11s 438us/step - loss: 0.5999 - acc: 0.6956 - val_loss: 0.5655 - val_acc: 0.7168

Epoch 00001: val_loss improved from inf to 0.56552, saving model to ./빅콘테스트/model/version2/mlp_leave_clf.hdf5
Epoch 2/150
24000/24000 [==============================] - 1s 60us/step - loss: 0.5520 - acc: 0.7231 - val_loss: 0.5634 - val_acc: 0.7192

Epoch 00002: val_loss improved from 0.56552 to 0.56339, saving model to ./빅콘테스트/model/version2/mlp_leave_clf.hdf5
Epoch 3/150
24000/24000 [==============================] - 1s 60us/step - loss: 0.5352 - acc: 0.7308 - val_loss: 0.5537 - val_acc: 0.7287

Epoch 00003: val_loss improved from 0.56339 to 0.55365, saving model to ./빅콘테스트/model/version2/mlp_leave_clf.hdf5
Epoch 4/150
24000/24000 [==============================] - 1s 58us/step - loss: 0.5227 - acc: 0.7382 - val_loss: 0.5861 - val_acc: 0.7263

Epoch 00004: val_loss did not improve from 0.55365
Epoc

## 결제 여부

### AdaBoost

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state = 42)

param_grid = [
    {'max_depth' : [7, 8, 9, 10]}
  ]

grid_search = GridSearchCV(tree, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['amount'])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  20 | elapsed:    1.6s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1, param_grid=[{'max_depth': [7, 8, 9, 10]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=

In [41]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8415333333333334
{'max_depth': 7}


In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 7), random_state = 42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'n_estimators' : [300, 500, 750]}
  ]

grid_search = GridSearchCV(ada, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['amount'])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  45 | elapsed:  4.7min remaining: 37.3min
[Parallel(n_jobs=-1)]: Done  28 out of  45 | elapsed: 10.5min remaining:  6.4min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 15.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                                criterion='gini',
                                                                                max_depth=7,
                                                                                max_features=None,
                                                                                max_leaf_nodes=None,
                                                                                min_impurity_decrease=0.0,
                                                                                min_impurity_split=None,
                                                                                min_samples_leaf=1,
                                                                                min

In [43]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8504333333333334
{'learning_rate': 0.01, 'n_estimators': 300}


In [44]:
# ada_leave_clf = AdaBoostClassifier(random_state = 42, base_estimator = DecisionTreeClassifier(max_depth = 9),
#                                    n_estimators = 500, learning_rate = 0.001)
# ada_amount_clf.fit(X_train, y_train['amount'])
ada_amount_clf = grid_search.best_estimator_
joblib.dump(ada_amount_clf, './빅콘테스트/model/version2/ada_amount_clf_version2.pkl')

['./빅콘테스트/model/version2/ada_amount_clf_version2.pkl']

### GradientBoost

In [45]:
gb = GradientBoostingClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01],
     'max_depth' : [8, 9],
     'n_estimators' : [300, 500]}
  ]

grid_search = GridSearchCV(gb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['amount'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  19 out of  40 | elapsed: 13.0min remaining: 14.4min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 21.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 21.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort=

In [46]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8562333333333333
{'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 500}


In [47]:
# gb_amount_clf = GradientBoostingClassifier(random_state = 42, learning_rate = 0.001, max_depth = 9, n_estimators = 300)
# gb_amount_clf.fit(X_train, y_train['amount'])
gb_amount_clf = grid_search.best_estimator_
joblib.dump(gb_amount_clf, './빅콘테스트/model/version2/gb_amount_clf_version2.pkl')

['./빅콘테스트/model/version2/gb_amount_clf_version2.pkl']

### RandomForest

In [48]:
rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train['amount'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.7min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [49]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8585333333333334
{'max_features': 15, 'n_estimators': 500}


In [50]:
# rf_amount_clf = RandomForestClassifier(random_state = 42, max_features = 19, n_estimators = 750)
# rf_amount_clf.fit(X_train, y_train['amount'])
rf_amount_clf = grid_search.best_estimator_
joblib.dump(rf_amount_clf, './빅콘테스트/model/version2/rf_amount_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_amount_clf_version2.pkl']

In [51]:
Importance = pd.DataFrame({'feature' : X_train_columns[1:], 'importance' : rf_amount_clf.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
49,enchant_count_1,0.000045
50,enchant_count_2,0.000057
52,enchant_count_4,0.000104
83,server_group_0,0.000107
85,server_group_2,0.000222
51,enchant_count_3,0.000245
84,server_group_1,0.000249
71,combat_same_pledge_cnt_3,0.000270
72,combat_same_pledge_cnt_4,0.000365
57,combat_random_attacker_cnt_1,0.000385


### XGBoost

In [52]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train, y_train['amount'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  4.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bytree=1,
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_job

In [53]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8575333333333334
{'gamma': 3, 'learning_rate': 0.1, 'max_depth': 11}


In [54]:
# xgb_amount_clf = XGBClassifier(random_state = 42, eta = 0.1, gamma = 1, max_depth = 11)
# xgb_amount_clf.fit(X_train, y_train['amount'])
xgb_amount_clf = grid_search.best_estimator_
joblib.dump(xgb_amount_clf, './빅콘테스트/model/version2/xgb_amount_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_amount_clf_version2.pkl']

In [55]:
Importance = pd.DataFrame({'feature' : X_train_columns[1:], 'importance' : xgb_amount_clf.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
52,enchant_count_4,0.000000
83,server_group_0,0.000000
85,server_group_2,0.000000
49,enchant_count_1,0.000068
71,combat_same_pledge_cnt_3,0.000068
50,enchant_count_2,0.000068
51,enchant_count_3,0.000136
33,exp_recovery_1,0.000136
70,combat_same_pledge_cnt_2,0.000204
36,exp_recovery_4,0.000204


### Ensemble

In [56]:
voting_amount_clf_1 = VotingClassifier(
    estimators=[('ada', ada_amount_clf), ('gb', gb_amount_clf), ('rf', rf_amount_clf), ('xgb', xgb_amount_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_amount_clf_1, X_train,
                           y_train['amount'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.8586

In [57]:
voting_amount_clf_2 = VotingClassifier(
    estimators=[('rf', rf_amount_clf), ('xgb', xgb_amount_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_amount_clf_2, X_train,
                           y_train['amount'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.8596666666666666

In [58]:
voting_amount_clf_1.fit(X_train, y_train['amount'])
joblib.dump(voting_amount_clf_1, './빅콘테스트/model/version2/voting_amount_clf_1_version2.pkl')

['./빅콘테스트/model/version2/voting_amount_clf_1_version2.pkl']

In [59]:
voting_amount_clf_2.fit(X_train, y_train['amount'])
joblib.dump(voting_amount_clf_2, './빅콘테스트/model/version2/voting_amount_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_amount_clf_2_version2.pkl']

### MLP(Multi Layer Perceptron)

In [60]:
input = Input(shape = (X_train.shape[1], ))

fc1 = Dense(1024, activation='relu')(input)
fc2 = Dense(512, activation='relu')(fc1)
fc3 = Dense(256, activation='relu')(fc2)
fc4 = Dense(128, activation='relu')(fc3)
fc4_drop = Dropout(0.1)(fc4)
fc5 = Dense(64, activation='relu')(fc4_drop)
fc5_drop = Dropout(0.1)(fc5)
fc6 = Dense(32, activation='relu')(fc5_drop)

amount_out = Dense(1, activation='sigmoid', name='amount_output')(fc6)

mlp_amount_clf = Model(inputs = input, outputs = amount_out)
mlp_amount_clf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 146)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              150528    
_________________________________________________________________
dense_8 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
__________

In [61]:
model_path = './빅콘테스트/model/version2/mlp_amount_clf.hdf5'

mlp_amount_clf.compile(optimizer = 'rmsprop', loss = {'amount_output' : 'binary_crossentropy'},
              metrics = {'amount_output' : 'acc'})

callbacks_list = [EarlyStopping(monitor = 'val_loss', patience = 20),
                 ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10),
                 ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)]

history = mlp_amount_clf.fit(X_train, y_train['amount'],
                             epochs = 150, batch_size = 128, 
                             callbacks = callbacks_list,
                             validation_split = 0.2,
                             shuffle = True)

Train on 24000 samples, validate on 6000 samples
Epoch 1/150
24000/24000 [==============================] - 2s 82us/step - loss: 0.4796 - acc: 0.7856 - val_loss: 0.4451 - val_acc: 0.8090

Epoch 00001: val_loss improved from inf to 0.44513, saving model to ./빅콘테스트/model/version2/mlp_amount_clf.hdf5
Epoch 2/150
24000/24000 [==============================] - 1s 62us/step - loss: 0.4068 - acc: 0.8280 - val_loss: 0.4604 - val_acc: 0.8233

Epoch 00002: val_loss did not improve from 0.44513
Epoch 3/150
24000/24000 [==============================] - 2s 65us/step - loss: 0.3902 - acc: 0.8377 - val_loss: 0.4078 - val_acc: 0.8327

Epoch 00003: val_loss improved from 0.44513 to 0.40779, saving model to ./빅콘테스트/model/version2/mlp_amount_clf.hdf5
Epoch 4/150
24000/24000 [==============================] - 1s 62us/step - loss: 0.3776 - acc: 0.8452 - val_loss: 0.4166 - val_acc: 0.8327

Epoch 00004: val_loss did not improve from 0.40779
Epoch 5/150
24000/24000 [==============================] - 1s 61us/

# 3. 테스트

## 잔존 여부

In [62]:
X_test = pd.read_csv('X_test_ver2.csv')

In [63]:
# X_test = minMaxScaler.transform(X_test.iloc[:, 1:])
X_test = X_test.iloc[:, 1:]

### AdaBoost

In [64]:
y_pred_ada = ada_leave_clf.predict(X_test)
y_pred_ada

array([0, 0, 0, ..., 1, 0, 0])

In [65]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['leave'], y_pred_ada))
print('recall :', recall_score(y_test['leave'], y_pred_ada))
confusion_matrix(y_test['leave'], y_pred_ada)

acc : 0.7543
recall : 0.6758498111530771


array([[4501,  998],
       [1459, 3042]])

### GradintBoost

In [66]:
y_pred_gb = gb_leave_clf.predict(X_test)
y_pred_gb

array([0, 0, 0, ..., 0, 0, 0])

In [67]:
print('acc :', accuracy_score(y_test['leave'], y_pred_gb))
print('recall :', recall_score(y_test['leave'], y_pred_gb))
confusion_matrix(y_test['leave'], y_pred_gb)

acc : 0.7575
recall : 0.6949566762941568


array([[4447, 1052],
       [1373, 3128]])

### RandomForest

In [68]:
y_pred_rf = rf_leave_clf.predict(X_test)
y_pred_rf

array([0, 0, 0, ..., 0, 0, 0])

In [69]:
print('acc :', accuracy_score(y_test['leave'], y_pred_rf))
print('recall :', recall_score(y_test['leave'], y_pred_rf))
confusion_matrix(y_test['leave'], y_pred_rf)

acc : 0.7681
recall : 0.70228838035992


array([[4520,  979],
       [1340, 3161]])

### XGBoost

In [70]:
y_pred_xgb = xgb_leave_clf.predict(X_test)
y_pred_xgb

array([0, 0, 0, ..., 0, 0, 0])

In [71]:
print('acc :', accuracy_score(y_test['leave'], y_pred_xgb))
print('recall :', recall_score(y_test['leave'], y_pred_xgb))
confusion_matrix(y_test['leave'], y_pred_xgb)

acc : 0.7589
recall : 0.704287936014219


array([[4419, 1080],
       [1331, 3170]])

### Ensemble

In [72]:
y_pred_ensemble_1 = voting_leave_clf_1.predict(X_test)
y_pred_ensemble_1

array([0, 0, 0, ..., 0, 0, 0])

In [73]:
print('acc :', accuracy_score(y_test['leave'], y_pred_ensemble_1))
print('recall :', recall_score(y_test['leave'], y_pred_ensemble_1))
confusion_matrix(y_test['leave'], y_pred_ensemble_1)

acc : 0.7673
recall : 0.7002888247056209


array([[4521,  978],
       [1349, 3152]])

In [74]:
y_pred_ensemble_2 = voting_leave_clf_2.predict(X_test)
y_pred_ensemble_2

array([0, 0, 0, ..., 0, 0, 0])

In [75]:
print('acc :', accuracy_score(y_test['leave'], y_pred_ensemble_2))
print('recall :', recall_score(y_test['leave'], y_pred_ensemble_2))
confusion_matrix(y_test['leave'], y_pred_ensemble_2)

acc : 0.7671
recall : 0.7040657631637414


array([[4502,  997],
       [1332, 3169]])

### MLP(Multi Layer Perceptron)

In [76]:
y_pred_mlp = mlp_leave_clf.predict(X_test)
y_pred_mlp = np.where(y_pred_mlp >= 0.5, 1, 0)

In [77]:
print('acc :', accuracy_score(y_test['leave'], y_pred_mlp))
print('recall :', recall_score(y_test['leave'], y_pred_mlp))
confusion_matrix(y_test['leave'], y_pred_mlp)

acc : 0.7074
recall : 0.6551877360586537


array([[4125, 1374],
       [1552, 2949]])

## 결제 여부

### AdaBoost

In [78]:
y_pred_ada = ada_amount_clf.predict(X_test)
y_pred_ada

array([1, 1, 1, ..., 1, 0, 1])

In [79]:
print('acc :', accuracy_score(y_test['amount'], y_pred_ada))
print('recall :', recall_score(y_test['amount'], y_pred_ada))
confusion_matrix(y_test['amount'], y_pred_ada)

acc : 0.851
recall : 0.9043358568479009


array([[3254,  934],
       [ 556, 5256]])

### GradientBoost

In [80]:
y_pred_gb = gb_amount_clf.predict(X_test)
y_pred_gb

array([1, 1, 1, ..., 1, 0, 1])

In [81]:
print('acc :', accuracy_score(y_test['amount'], y_pred_gb))
print('recall :', recall_score(y_test['amount'], y_pred_gb))
confusion_matrix(y_test['amount'], y_pred_gb)

acc : 0.8559
recall : 0.9077770130763937


array([[3283,  905],
       [ 536, 5276]])

### RandomForest

In [82]:
y_pred_rf = rf_amount_clf.predict(X_test)
y_pred_rf

array([1, 1, 1, ..., 1, 0, 1])

In [83]:
print('acc :', accuracy_score(y_test['amount'], y_pred_rf))
print('recall :', recall_score(y_test['amount'], y_pred_rf))
confusion_matrix(y_test['amount'], y_pred_rf)

acc : 0.8586
recall : 0.9230901582931865


array([[3221,  967],
       [ 447, 5365]])

### XGBoost

In [84]:
y_pred_xgb = xgb_amount_clf.predict(X_test)
y_pred_xgb

array([1, 1, 1, ..., 1, 0, 1])

In [85]:
print('acc :', accuracy_score(y_test['amount'], y_pred_xgb))
print('recall :', recall_score(y_test['amount'], y_pred_xgb))
confusion_matrix(y_test['amount'], y_pred_xgb)

acc : 0.8568
recall : 0.9070887818306951


array([[3296,  892],
       [ 540, 5272]])

### Ensemble

In [86]:
y_pred_ensemble_1 = voting_amount_clf_1.predict(X_test)
y_pred_ensemble_1

array([1, 1, 1, ..., 1, 0, 1])

In [87]:
print('acc :', accuracy_score(y_test['amount'], y_pred_ensemble_1))
print('recall :', recall_score(y_test['amount'], y_pred_ensemble_1))
confusion_matrix(y_test['amount'], y_pred_ensemble_1)

acc : 0.8572
recall : 0.9124225739848589


array([[3269,  919],
       [ 509, 5303]])

In [88]:
y_pred_ensemble_2 = voting_amount_clf_2.predict(X_test)
y_pred_ensemble_2

array([1, 1, 1, ..., 1, 0, 1])

In [89]:
print('acc :', accuracy_score(y_test['amount'], y_pred_ensemble_2))
print('recall :', recall_score(y_test['amount'], y_pred_ensemble_2))
confusion_matrix(y_test['amount'], y_pred_ensemble_2)

acc : 0.858
recall : 0.9151754989676532


array([[3261,  927],
       [ 493, 5319]])

### MLP(Multi Layer Perceptron)

In [90]:
y_pred_mlp = mlp_amount_clf.predict(X_test)
y_pred_mlp = np.where(y_pred_mlp >= 0.5, 1, 0)

In [91]:
print('acc :', accuracy_score(y_test['amount'], y_pred_mlp))
print('recall :', recall_score(y_test['amount'], y_pred_mlp))
confusion_matrix(y_test['amount'], y_pred_mlp)

acc : 0.823
recall : 0.85719201651755


array([[3248,  940],
       [ 830, 4982]])

# non class

In [92]:
X_train_non_class = pd.read_csv('X_train_ver2_non_class.csv')

## 생존 여부

### RandomForest

In [94]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_non_class.iloc[:, 1:], y_train['leave'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.8min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [95]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7624
{'max_features': 15, 'n_estimators': 750}


In [96]:
from sklearn.externals import joblib

# rf_leave_non_class_clf = RandomForestClassifier(random_state = 42, max_features = 19, n_estimators = 750)
# rf_leave_non_class_clf.fit(X_train_non_class.iloc[:, 1:], y_train['leave'])
rf_leave_non_class_clf = grid_search.best_estimator_
joblib.dump(rf_leave_non_class_clf, './빅콘테스트/model/version2/rf_leave_non_class_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_leave_non_class_clf_version2.pkl']

### XGBoost

In [97]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_non_class.iloc[:, 1:], y_train['leave'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  4.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bytree=1,
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_job

In [98]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7558666666666667
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 12}


In [99]:
from sklearn.externals import joblib

# xgb_leave_non_class_clf = XGBClassifier(random_state = 42, eta = 0.1, gamma = 1, max_depth = 11)
# xgb_leave_non_class_clf.fit(X_train_non_class.iloc[:, 1:], y_train['leave'])
xgb_leave_non_class_clf = grid_search.best_estimator_
joblib.dump(xgb_leave_non_class_clf, './빅콘테스트/model/version2/xgb_leave_non_class_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_leave_non_class_clf_version2.pkl']

### Ensemble

In [100]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

voting_leave_non_class_clf_2 = VotingClassifier(
    estimators=[('rf', rf_leave_non_class_clf), ('xgb', xgb_leave_non_class_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_leave_non_class_clf_2, X_train_non_class.iloc[:, 1:],
                           y_train['leave'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.7612336169018598

In [101]:
voting_leave_non_class_clf_2.fit(X_train_non_class.iloc[:, 1:], y_train['leave'])
joblib.dump(voting_leave_non_class_clf_2, './빅콘테스트/model/version2/voting_leave_non_class_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_leave_non_class_clf_2_version2.pkl']

## 결제 여부

### RandomForest

In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_non_class.iloc[:, 1:], y_train['amount'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.7min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [103]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8585333333333334
{'max_features': 13, 'n_estimators': 500}


In [104]:
from sklearn.externals import joblib

# rf_amount_non_class_clf = RandomForestClassifier(random_state = 42, max_features = 19, n_estimators = 750)
# rf_amount_non_class_clf.fit(X_train_non_class.iloc[:, 1:], y_train['amount'])
rf_amount_non_class_clf = grid_search.best_estimator_
joblib.dump(rf_amount_non_class_clf, './빅콘테스트/model/version2/rf_amount_non_class_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_amount_non_class_clf_version2.pkl']

### XGBoost

In [105]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_non_class.iloc[:, 1:], y_train['amount'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  4.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bytree=1,
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_job

In [106]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8573333333333333
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 10}


In [107]:
from sklearn.externals import joblib

# xgb_amount_non_class_clf = XGBClassifier(random_state = 42, eta = 0.1, gamma = 1, max_depth = 11)
# xgb_amount_non_class_clf.fit(X_train_non_class.iloc[:, 1:], y_train['amount'])
xgb_amount_non_class_clf = grid_search.best_estimator_
joblib.dump(xgb_amount_non_class_clf, './빅콘테스트/model/version2/xgb_amount_non_class_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_amount_non_class_clf_version2.pkl']

### Ensemble

In [108]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

voting_amount_non_class_clf_2 = VotingClassifier(
    estimators=[('rf', rf_amount_non_class_clf), ('xgb', xgb_amount_non_class_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_amount_non_class_clf_2, X_train_non_class.iloc[:, 1:],
                           y_train['amount'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.8593

In [109]:
voting_amount_non_class_clf_2.fit(X_train_non_class.iloc[:, 1:], y_train['amount'])
joblib.dump(voting_amount_non_class_clf_2, './빅콘테스트/model/version2/voting_amount_non_class_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_amount_non_class_clf_2_version2.pkl']

# 테스트

In [111]:
X_test_non_class = pd.read_csv('X_test_ver2_non_class.csv')

## 잔존 여부

### RandomForest

In [112]:
y_pred_rf = rf_leave_non_class_clf.predict(X_test_non_class.iloc[:, 1:])
y_pred_rf

array([0, 0, 0, ..., 0, 0, 0])

In [113]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['leave'], y_pred_rf))
print('recall :', recall_score(y_test['leave'], y_pred_rf))
confusion_matrix(y_test['leave'], y_pred_rf)

acc : 0.767
recall : 0.6987336147522772


array([[4525,  974],
       [1356, 3145]])

### XGBoost

In [114]:
y_pred_xgb = xgb_leave_non_class_clf.predict(X_test_non_class.iloc[:, 1:])
y_pred_xgb

array([0, 0, 0, ..., 0, 0, 0])

In [115]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['leave'], y_pred_xgb))
print('recall :', recall_score(y_test['leave'], y_pred_xgb))
confusion_matrix(y_test['leave'], y_pred_xgb)

acc : 0.7616
recall : 0.7047322817151744


array([[4444, 1055],
       [1329, 3172]])

### Ensemble

In [116]:
y_pred_ensemble_2 = voting_leave_non_class_clf_2.predict(X_test_non_class.iloc[:, 1:])
y_pred_ensemble_2

array([0, 0, 0, ..., 0, 0, 0])

In [117]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['leave'], y_pred_ensemble_2))
print('recall :', recall_score(y_test['leave'], y_pred_ensemble_2))
confusion_matrix(y_test['leave'], y_pred_ensemble_2)

acc : 0.7657
recall : 0.7069540102199511


array([[4475, 1024],
       [1319, 3182]])

## 결제 여부

### RandomForest

In [118]:
y_pred_rf = rf_amount_non_class_clf.predict(X_test_non_class.iloc[:, 1:])
y_pred_rf

array([1, 1, 1, ..., 1, 0, 1])

In [119]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['amount'], y_pred_rf))
print('recall :', recall_score(y_test['amount'], y_pred_rf))
confusion_matrix(y_test['amount'], y_pred_rf)

acc : 0.8585
recall : 0.9230901582931865


array([[3220,  968],
       [ 447, 5365]])

### XGBoost

In [120]:
y_pred_xgb = xgb_amount_non_class_clf.predict(X_test_non_class.iloc[:, 1:])
y_pred_xgb

array([1, 1, 1, ..., 1, 0, 1])

In [121]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['amount'], y_pred_xgb))
print('recall :', recall_score(y_test['amount'], y_pred_xgb))
confusion_matrix(y_test['amount'], y_pred_xgb)

acc : 0.8564
recall : 0.9091534755677908


array([[3280,  908],
       [ 528, 5284]])

### Ensemble

In [122]:
y_pred_ensemble_2 = voting_amount_non_class_clf_2.predict(X_test_non_class.iloc[:, 1:])
y_pred_ensemble_2

array([1, 1, 1, ..., 1, 0, 1])

In [123]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['amount'], y_pred_ensemble_2))
print('recall :', recall_score(y_test['amount'], y_pred_ensemble_2))
confusion_matrix(y_test['amount'], y_pred_ensemble_2)

acc : 0.8577
recall : 0.9170681348933242


array([[3247,  941],
       [ 482, 5330]])

# 플레이 시간 변동

In [599]:
X_train_fluctuation = pd.read_csv('X_train_ver2_fluctuation.csv')

## 이탈 여부

### RandomForest

In [600]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

# 12, 13, 14, 15
param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  3.3min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  5.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [601]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7673666666666666
{'max_features': 15, 'n_estimators': 500}


In [602]:
from sklearn.externals import joblib

# rf_leave_fluctuation_clf = RandomForestClassifier(random_state = 42, max_features = 15, n_estimators = 500)
# rf_leave_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])
rf_leave_fluctuation_clf = grid_search.best_estimator_
joblib.dump(rf_leave_fluctuation_clf, './빅콘테스트/model/version2/rf_leave_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_leave_fluctuation_clf_version2.pkl']

### XGBoost

In [603]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

# 9, 10, 11, 12
param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bytree=1,
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_job

In [604]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7608333333333334
{'gamma': 1, 'learning_rate': 0.1, 'max_depth': 10}


In [605]:
from sklearn.externals import joblib

# xgb_leave_fluctuation_clf = XGBClassifier(random_state = 42, learning_rate = 0.1, gamma = 1, max_depth = 10)
# xgb_leave_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])
xgb_leave_fluctuation_clf = grid_search.best_estimator_
joblib.dump(xgb_leave_fluctuation_clf, './빅콘테스트/model/version2/xgb_leave_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_leave_fluctuation_clf_version2.pkl']

### Ensemble

In [606]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

voting_leave_fluctuation_clf_2 = VotingClassifier(
    estimators=[('rf', rf_leave_fluctuation_clf), ('xgb', xgb_leave_fluctuation_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_leave_fluctuation_clf_2, X_train_fluctuation.iloc[:, 1:],
                           y_train['leave'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.7662334559324109

In [607]:
voting_leave_fluctuation_clf_2.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])
joblib.dump(voting_leave_fluctuation_clf_2, './빅콘테스트/model/version2/voting_leave_fluctuation_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_leave_fluctuation_clf_2_version2.pkl']

## 결제 여부

### RandomForest

In [608]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.8min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [609]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8588666666666667
{'max_features': 12, 'n_estimators': 750}


In [610]:
from sklearn.externals import joblib

# rf_amount_fluctuation_clf = RandomForestClassifier(random_state = 42, max_features = 12, n_estimators = 750)
# rf_amount_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])
rf_amount_fluctuation_clf = grid_search.best_estimator_
joblib.dump(rf_amount_fluctuation_clf, './빅콘테스트/model/version2/rf_amount_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_amount_fluctuation_clf_version2.pkl']

### XGBoost

In [611]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.7min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bytree=1,
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_job

In [612]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8570333333333333
{'gamma': 1, 'learning_rate': 0.1, 'max_depth': 9}


In [613]:
from sklearn.externals import joblib

# xgb_amount_fluctuation_clf = XGBClassifier(random_state = 42, learning_rate = 0.1, gamma = 1, max_depth = 9)
# xgb_amount_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])
xgb_amount_fluctuation_clf = grid_search.best_estimator_
joblib.dump(xgb_amount_fluctuation_clf, './빅콘테스트/model/version2/xgb_amount_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_amount_fluctuation_clf_version2.pkl']

### Ensemble

In [614]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

voting_amount_fluctuation_clf_2 = VotingClassifier(
    estimators=[('rf', rf_amount_fluctuation_clf), ('xgb', xgb_amount_fluctuation_clf)],
    voting='soft', n_jobs = -1)

cv_value = cross_val_score(voting_amount_fluctuation_clf_2, X_train_fluctuation.iloc[:, 1:],
                           y_train['amount'], cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

0.8594333333333333

In [615]:
voting_amount_fluctuation_clf_2.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])
joblib.dump(voting_amount_fluctuation_clf_2, './빅콘테스트/model/version2/voting_amount_fluctuation_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_amount_fluctuation_clf_2_version2.pkl']

# 테스트

In [637]:
X_test_fluctuation = pd.read_csv('X_test_ver2_fluctuation.csv')

## 잔존 여부

### RandomForest

In [638]:
y_pred_rf = rf_leave_fluctuation_clf.predict(X_test_fluctuation.iloc[:, 1:])
y_pred_rf

array([0, 0, 0, ..., 0, 0, 0])

In [639]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['leave'], y_pred_rf))
print('recall :', recall_score(y_test['leave'], y_pred_rf))
confusion_matrix(y_test['leave'], y_pred_rf)

acc : 0.7711
recall : 0.7051766274161297


array([[4537,  962],
       [1327, 3174]])

### XGBoost

In [640]:
y_pred_xgb = xgb_leave_fluctuation_clf.predict(X_test_fluctuation.iloc[:, 1:])
y_pred_xgb

array([0, 0, 0, ..., 0, 0, 0])

In [641]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['leave'], y_pred_xgb))
print('recall :', recall_score(y_test['leave'], y_pred_xgb))
confusion_matrix(y_test['leave'], y_pred_xgb)

acc : 0.7648
recall : 0.7105087758275939


array([[4450, 1049],
       [1303, 3198]])

### Ensemble

In [642]:
y_pred_ensemble_2 = voting_leave_fluctuation_clf_2.predict(X_test_fluctuation.iloc[:, 1:])
y_pred_ensemble_2

array([0, 0, 0, ..., 0, 0, 0])

In [643]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['leave'], y_pred_ensemble_2))
print('recall :', recall_score(y_test['leave'], y_pred_ensemble_2))
confusion_matrix(y_test['leave'], y_pred_ensemble_2)

acc : 0.7706
recall : 0.7096200844256831


array([[4512,  987],
       [1307, 3194]])

## 결제 여부

### RandomForest

In [644]:
y_pred_rf = rf_amount_fluctuation_clf.predict(X_test_fluctuation.iloc[:, 1:])
y_pred_rf

array([1, 1, 1, ..., 1, 0, 1])

In [645]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['amount'], y_pred_rf))
print('recall :', recall_score(y_test['amount'], y_pred_rf))
confusion_matrix(y_test['amount'], y_pred_rf)

acc : 0.8564
recall : 0.9210254645560908


array([[3211,  977],
       [ 459, 5353]])

### XGBoost

In [646]:
y_pred_xgb = xgb_amount_fluctuation_clf.predict(X_test_fluctuation.iloc[:, 1:])
y_pred_xgb

array([1, 1, 1, ..., 1, 0, 1])

In [647]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['amount'], y_pred_xgb))
print('recall :', recall_score(y_test['amount'], y_pred_xgb))
confusion_matrix(y_test['amount'], y_pred_xgb)

acc : 0.8556
recall : 0.9110461114934618


array([[3261,  927],
       [ 517, 5295]])

### Ensemble

In [648]:
y_pred_ensemble_2 = voting_amount_fluctuation_clf_2.predict(X_test_fluctuation.iloc[:, 1:])
y_pred_ensemble_2

array([1, 1, 1, ..., 1, 0, 1])

In [649]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

print('acc :', accuracy_score(y_test['amount'], y_pred_ensemble_2))
print('recall :', recall_score(y_test['amount'], y_pred_ensemble_2))
confusion_matrix(y_test['amount'], y_pred_ensemble_2)

acc : 0.8569
recall : 0.9179284239504474


array([[3234,  954],
       [ 477, 5335]])

# 교차검증 X

In [695]:
X_train_fluctuation = pd.read_csv('X_train_ver2_fluctuation.csv')

In [675]:
np.random.seed(42)
idx = np.random.permutation(30000)

y_train.index = range(0, len(y_train))
X_train_fluctuation_shuffle = X_train_fluctuation.loc[idx]
y_train_shuffle = y_train.loc[idx]

X_train_fluctuation_1 = X_train_fluctuation_shuffle[:15000]
X_train_fluctuation_2 = X_train_fluctuation_shuffle[15000:]
y_train_1 = y_train_shuffle[:15000]
y_train_2 = y_train_shuffle[15000:]

## 이탈 여부

In [697]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

rf_leave_fluctuation_clf = RandomForestClassifier(random_state = 42, max_features = 15, n_estimators = 500)
rf_leave_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])
joblib.dump(rf_leave_fluctuation_clf, './빅콘테스트/model/version2/rf_leave_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_leave_fluctuation_clf_version2.pkl']

In [698]:
from xgboost import XGBClassifier
from sklearn.externals import joblib

xgb_leave_fluctuation_clf = XGBClassifier(random_state = 42, learning_rate = 0.1, gamma = 1, max_depth = 10)
xgb_leave_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])
joblib.dump(xgb_leave_fluctuation_clf, './빅콘테스트/model/version2/xgb_leave_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_leave_fluctuation_clf_version2.pkl']

In [699]:
from sklearn.ensemble import VotingClassifier
from sklearn.externals import joblib

voting_leave_fluctuation_clf_2 = VotingClassifier(
    estimators=[('rf', rf_leave_fluctuation_clf), ('xgb', xgb_leave_fluctuation_clf)],
    voting='soft', n_jobs = -1)

voting_leave_fluctuation_clf_2.fit(X_train_fluctuation.iloc[:, 1:], y_train['leave'])
joblib.dump(voting_leave_fluctuation_clf_2, './빅콘테스트/model/version2/voting_leave_fluctuation_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_leave_fluctuation_clf_2_version2.pkl']

## 결제 여부

In [700]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

rf_amount_fluctuation_clf = RandomForestClassifier(random_state = 42, max_features = 12, n_estimators = 750)
rf_amount_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])
joblib.dump(rf_amount_fluctuation_clf, './빅콘테스트/model/version2/rf_amount_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/rf_amount_fluctuation_clf_version2.pkl']

In [701]:
from xgboost import XGBClassifier
from sklearn.externals import joblib

xgb_amount_fluctuation_clf = XGBClassifier(random_state = 42, learning_rate = 0.1, gamma = 1, max_depth = 9)
xgb_amount_fluctuation_clf.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])
joblib.dump(xgb_amount_fluctuation_clf, './빅콘테스트/model/version2/xgb_amount_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/xgb_amount_fluctuation_clf_version2.pkl']

In [702]:
from sklearn.ensemble import VotingClassifier
from sklearn.externals import joblib

voting_amount_fluctuation_clf_2 = VotingClassifier(
    estimators=[('rf', rf_amount_fluctuation_clf), ('xgb', xgb_amount_fluctuation_clf)],
    voting='soft', n_jobs = -1)

voting_amount_fluctuation_clf_2.fit(X_train_fluctuation.iloc[:, 1:], y_train['amount'])
joblib.dump(voting_amount_fluctuation_clf_2, './빅콘테스트/model/version2/voting_amount_fluctuation_clf_2_version2.pkl')

['./빅콘테스트/model/version2/voting_amount_fluctuation_clf_2_version2.pkl']

## 스태킹

In [687]:
leave_pred1 = rf_leave_fluctuation_clf.predict_proba(X_train_fluctuation_2.iloc[:, 1:])
leave_pred2 = xgb_leave_fluctuation_clf.predict_proba(X_train_fluctuation_2.iloc[:, 1:])
leave_pred3 = voting_leave_fluctuation_clf_2.predict_proba(X_train_fluctuation_2.iloc[:, 1:])

leave_pred = np.concatenate([leave_pred1, leave_pred2, leave_pred3], axis = 1)

In [688]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state = 42)

param_grid = [
    {'penalty' : ['l1', 'l2'], 'C' : [0.1, 1, 3]}
  ]

grid_search = GridSearchCV(lr, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(leave_pred, y_train_2['leave'])

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.6s finished
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 3], 'penalty': ['l1', 'l2']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=2)

In [689]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7598
{'C': 0.1, 'penalty': 'l2'}


In [690]:
lr_leave_fluctuation_clf = grid_search.best_estimator_
joblib.dump(lr_leave_fluctuation_clf, './빅콘테스트/model/version2/lr_leave_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/lr_leave_fluctuation_clf_version2.pkl']

In [691]:
amount_pred1 = rf_amount_fluctuation_clf.predict_proba(X_train_fluctuation_2.iloc[:, 1:])
amount_pred2 = xgb_amount_fluctuation_clf.predict_proba(X_train_fluctuation_2.iloc[:, 1:])
amount_pred3 = voting_amount_fluctuation_clf_2.predict_proba(X_train_fluctuation_2.iloc[:, 1:])

amount_pred = np.concatenate([amount_pred1, amount_pred2, amount_pred3], axis = 1)

In [692]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state = 42)

param_grid = [
    {'penalty' : ['l1', 'l2'], 'C' : [0.1, 1, 3]}
  ]

grid_search = GridSearchCV(lr, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(amount_pred, y_train_2['amount'])

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.2s finished
/home/user3/.conda/envs/hun/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 3], 'penalty': ['l1', 'l2']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=2)

In [693]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.9998666666666667
{'C': 0.1, 'penalty': 'l1'}


In [694]:
lr_amount_fluctuation_clf = grid_search.best_estimator_
joblib.dump(lr_amount_fluctuation_clf, './빅콘테스트/model/version2/lr_amount_fluctuation_clf_version2.pkl')

['./빅콘테스트/model/version2/lr_amount_fluctuation_clf_version2.pkl']